## Maximal Matching Tokenizer

In [1]:
import re
from collections import defaultdict
from heapq import heappush, heappop  # for priority queue
from marisa_trie import Trie

# from khmernlp.tokenize import DEFAULT_WORD_DICT_TRIE




wordlist = [li.strip() for li in open('./khmer_words.txt', 'r', encoding='utf-8')]
trie = Trie(wordlist)

# trie = DEFAULT_WORD_DICT_TRIE

# ช่วยตัดพวกภาษาอังกฤษ เป็นต้น
pat_eng = re.compile(r'''(?x)
[-a-zA-Z]+|   # english
\d[\d,\.]*|   # number
[ \t]+|       # space
\r?\n         # newline
'''
' '
)



def onecut(text):
  words_at = defaultdict(list)  # main data structure

  def serialize(p, p2):    # helper function
    for w in words_at[p]:
      p_ = p + len(w)
      if p_== p2:
        yield [w]
      elif p_ < p2:
        for path in serialize(p_, p2):
          yield [w]+path

  q = [0]       # min-heap queue
  last_p = 0    # last position for yield
  while q[0] < len(text):
      p = heappop(q)

      for w in trie.prefixes(text[p:]):
          words_at[p].append(w)
          if p+len(w) not in q:
            heappush(q, p+len(w))

      if len(q)==1:
          for w in min(serialize(last_p, q[0]), key=len):
            yield w
          last_p = q[0]

      if len(q)==0:
          m = pat_eng.match(text[p:])
          if m:
              i = p + m.span()[1]
          else:
              for i in range(p, len(text)):
                  ww = trie.prefixes(text[i:])
                  m = pat_eng.match(text[i:])
                  if ww or m:
                      break
              else:
                  i = len(text)
          w = text[p:i]
          words_at[p].append(w)
          yield w
          last_p = i
          heappush(q, i)

In [2]:
text = """

ជារឿងគួរឱ្យរំភើប បច្ចេកវិទ្យាតាមដានទឹកជំនន់ច្នៃបង្កើតឡើងដោយក្រុមហ៊ុន ArrowDot មានមូលដ្ឋាននៅកម្ពុជា មានស្ថាបនិក


"""

## Compare result Maximal matching tokenizer vs Khmercut

In [3]:
print(list(onecut(text)))

list_words = list(onecut(text))

ignore_words = ['\n', ' ', '  ', '\u200b', '\t', '\r']

for word in list_words:
    if word not in ignore_words:
        print(word)



['\n', '\n', 'ជា', 'រឿង', 'គួរ', 'ឱ្យ', 'រំភើប', ' ', 'បច្ចេកវិទ្យា', 'តាមដាន', 'ទឹកជំនន់', 'ច្នៃ', 'បង្កើតឡើង', 'ដោយ', 'ក្រុមហ៊ុន', ' ', 'ArrowDot', ' ', 'មាន', 'មូលដ្ឋាន', 'នៅ', 'កម្ពុជា', ' ', 'មាន', 'ស្ថាបនិក', '\n', '\n', '\n']
ជា
រឿង
គួរ
ឱ្យ
រំភើប
បច្ចេកវិទ្យា
តាមដាន
ទឹកជំនន់
ច្នៃ
បង្កើតឡើង
ដោយ
ក្រុមហ៊ុន
ArrowDot
មាន
មូលដ្ឋាន
នៅ
កម្ពុជា
មាន
ស្ថាបនិក


In [4]:
from khmercut import tokenize

tokenize(text)

# list_words_2 = list(tokenize(text))
# list_words_2




['\n\n',
 'ជា',
 'រឿង',
 'គួរឱ្យ',
 'រំភើប',
 ' ',
 'បច្ចេកវិទ្យា',
 'តាមដាន',
 'ទឹក',
 'ជំនន់',
 'ច្នៃ',
 'បង្កើតឡើង',
 'ដោយ',
 'ក្រុមហ៊ុន',
 ' ',
 'ArrowDot',
 ' ',
 'មាន',
 'មូលដ្ឋាន',
 'នៅ',
 'កម្ពុជា',
 ' ',
 'មាន',
 'ស្ថាបនិក',
 '\n\n\n']

#### Summary 
Overall khmercut outperform maximal matching

## Multicut

In [5]:
"""
Trie data structure.

Designed to be used for tokenizer's dictionary, but can be for other purposes.
"""
from typing import Iterable, Iterator, List, Union


class Trie(Iterable[str]):
    class Node:
        __slots__ = "end", "children"

        def __init__(self):
            self.end = False
            self.children = {}

    def __init__(self, words: Iterable[str]):
        self.words = set(words)
        self.root = Trie.Node()

        for word in words:
            self.add(word)

    def add(self, word: str) -> None:
        """
        Add a word to the trie.
        Spaces in front of and following the word will be removed.

        :param str text: a word
        """
        word = word.strip()
        self.words.add(word)
        cur = self.root
        for ch in word:
            child = cur.children.get(ch)
            if not child:
                child = Trie.Node()
                cur.children[ch] = child
            cur = child
        cur.end = True

    def remove(self, word: str) -> None:
        """
        Remove a word from the trie.
        If the word is not found, do nothing.

        :param str text: a word
        """
        # remove from set first
        if word not in self.words:
            return
        self.words.remove(word)
        # then remove from nodes
        parent = self.root
        data = []  # track path to leaf
        for ch in word:
            child = parent.children[ch]
            data.append((parent, child, ch))
            parent = child
        # remove the last one
        child.end = False
        # prune up the tree
        for parent, child, ch in reversed(data):
            if child.end or child.children:
                break
            del parent.children[ch]  # remove from parent dict

    def prefixes(self, text: str) -> List[str]:
        """
        List all possible words from first sequence of characters in a word.

        :param str text: a word
        :return: a list of possible words
        :rtype: List[str]
        """
        res = []
        cur = self.root
        for i, ch in enumerate(text):
            node = cur.children.get(ch)
            if not node:
                break
            if node.end:
                res.append(text[: i + 1])
            cur = node
        return res

    def __contains__(self, key: str) -> bool:
        return key in self.words

    def __iter__(self) -> Iterator[str]:
        yield from self.words

    def __len__(self) -> int:
        return len(self.words)



def dict_trie(dict_source: Union[str, Iterable[str], Trie]) -> Trie:
    """
    Create a dictionary trie from a file or an iterable.

    :param str|Iterable[str]|pythainlp.util.Trie dict_source: a path to
        dictionary file or a list of words or a pythainlp.util.Trie object
    :return: a trie object
    :rtype: pythainlp.util.Trie
    """
    trie = Trie([])

    if isinstance(dict_source, str) and len(dict_source) > 0:
        # dict_source is a path to dictionary text file
        with open(dict_source, "r", encoding="utf8") as f:
            _vocabs = f.read().splitlines()
            trie = Trie(_vocabs)
    elif isinstance(dict_source, Iterable) and not isinstance(
        dict_source, str
    ):
        # Note: Since Trie and str are both Iterable,
        # so the Iterable check should be here, at the very end,
        # because it has less specificality
        trie = Trie(dict_source)
    else:
        raise TypeError(
            "Type of dict_source must be pythainlp.util.Trie, "
            "or Iterable[str], or non-empty str (path to source file)"
        )

    return trie



In [6]:

from typing import FrozenSet, List, Union
import warnings


_KHMER_WORDS: FrozenSet[str] = frozenset()
_KHMER_WORDS_FILENAME = "khmer_words.txt"


def get_corpus(path: str, comments: bool = True) -> frozenset:
    path = path.strip()
    lines = []
    with open(path, "r", encoding="utf-8-sig") as fh:
        lines = fh.read().splitlines()

    if not comments:
        # if the line has a '#' character, take only text before the first '#'
        lines = [line.split("#", 1)[0].strip() for line in lines]

    return frozenset(filter(None, lines))

def khmer_words() -> FrozenSet[str]:
    """

    :return: :class:`frozenset` containing words in the Khmer language.
    :rtype: :class:`frozenset`
    """
    global _KHMER_WORDS
    if not _KHMER_WORDS:
        _KHMER_WORDS = get_corpus(_KHMER_WORDS_FILENAME)

    return _KHMER_WORDS

In [7]:

"""
Multi cut -- Thai word segmentation with maximum matching.
Original codes from Korakot Chaovavanich.

"""

import re
from collections import defaultdict
from typing import Iterator, List


DEFAULT_WORD_DICT_TRIE = Trie(khmer_words())
# DEFAULT_WORD_DICT_TRIE = Trie("./khmer_words.txt")



class LatticeString(str):
    """String that keeps possible tokenizations"""

    def __new__(cls, value, multi=None, in_dict=True):
        return str.__new__(cls, value)

    def __init__(self, value, multi=None, in_dict=True):
        self.unique = True
        if multi:
            self.multi = list(multi)
            if len(self.multi) > 1:
                self.unique = False
        else:
            self.multi = [value]
        self.in_dict = in_dict  # if in dictionary


_RE_NONKHMER = r"""(?x)
[-a-zA-Z]+|       # Latin characters
\d+([,\.]\d+)*|   # numbers
[ \t]+|           # spaces
\r?\n             # newlines
"""
_PAT_NONKHMER = re.compile(_RE_NONKHMER)


def _multicut(
    text: str, custom_dict: Trie = DEFAULT_WORD_DICT_TRIE
) -> Iterator[LatticeString]:
    """Return LatticeString"""
    if not custom_dict:
        custom_dict = DEFAULT_WORD_DICT_TRIE

    len_text = len(text)
    words_at = defaultdict(list)  # main data structure

    def serialize(p, p2):  # helper function
        for w in words_at[p]:
            p_ = p + len(w)
            if p_ == p2:
                yield w
            elif p_ < p2:
                for path in serialize(p_, p2):
                    yield w + "/" + path

    q = {0}
    last_p = 0  # last position for yield
    while min(q) < len_text:
        p = min(q)
        q -= {p}  # q.pop, but for set

        for w in custom_dict.prefixes(text[p:]):
            words_at[p].append(w)
            q.add(p + len(w))

        len_q = len(q)

        if len_q == 1:
            q0 = min(q)
            yield LatticeString(text[last_p:q0], serialize(last_p, q0))
            last_p = q0
        elif len_q == 0:  # len(q) == 0  means not found in dictionary
            m = _PAT_NONKHMER.match(text[p:])
            if m:  # non-Thai token
                i = p + m.span()[1]
            else:  # non-Thai token, find minimum skip
                for i in range(p, len_text):
                    ww = custom_dict.prefixes(text[i:])
                    m = _PAT_NONKHMER.match(text[i:])
                    if ww or m:
                        break
                else:
                    i = len_text
            w = text[p:i]
            words_at[p].append(w)
            yield LatticeString(w, in_dict=False)
            last_p = i
            q.add(i)


def mmcut(text: str) -> List[str]:
    res = []
    for w in _multicut(text):
        mm = min(w.multi, key=lambda x: x.count("/"))
        res.extend(mm.split("/"))
    return res


def _combine(ww: List[LatticeString]) -> Iterator[str]:
    if ww == []:
        yield ""
    else:
        w = ww[0]
        for tail in _combine(ww[1:]):
            if w.unique:
                yield w + "|" + tail
            else:
                for m in w.multi:
                    yield m.replace("/", "|") + "|" + tail


def segment(
    text: str, custom_dict: Trie = DEFAULT_WORD_DICT_TRIE
) -> List[str]:
    """Dictionary-based maximum matching word segmentation.

    :param text: text to be tokenized
    :type text: str
    :param custom_dict: tokenization dictionary,\
        defaults to DEFAULT_WORD_DICT_TRIE
    :type custom_dict: Trie, optional
    :return: list of segmented tokens
    :rtype: List[str]
    """
    if not text or not isinstance(text, str):
        return []

    return list(_multicut(text, custom_dict=custom_dict))


def find_all_segment(
    text: str, custom_dict: Trie = DEFAULT_WORD_DICT_TRIE
) -> List[str]:
    """Get all possible segment variations.

    :param text: input string to be tokenized
    :type text: str
    :param custom_dict: tokenization dictionary,\
        defaults to DEFAULT_WORD_DICT_TRIE
    :type custom_dict: Trie, optional
    :return: list of segment variations
    :rtype: List[str]
    """
    if not text or not isinstance(text, str):
        return []

    ww = list(_multicut(text, custom_dict=custom_dict))

    return list(_combine(ww))


In [8]:
text_all_segment = """

ជាការរចនាមួយដែលអាចឲ្យយើងពាក់ជាប់ជាមួយនឹងត្រចៀកដោយមិនមានការឈឺ ទទួលបានគុណភាពសំឡេងល្អ លើសពីនេះទៅទៀតពេលដែលពាក់កាស់នេះជាប់ត្រចៀក យើងនៅតែអាចឮអ្នកជុំវិញខ្លួននិយាយឬក៏ធ្វើការសន្ទនាជាមួយគ្នាយ៉ាងល្អ ហើយវាក៏ជួយរក្សាការសម្ងាត់នូវរាល់អ្វីដែលយើងបានស្តាប់មិនឲ្យឮចេញចេញក្រៅផងដែរ។ 

"""

In [9]:
# segment(text_all_segment)

In [10]:
# find_all_segment(text_all_segment)


In [11]:

"""
Generic functions of tokenizers
"""
import re
from typing import Iterable, List, Union


DEFAULT_WORD_TOKENIZE_ENGINE = "multi_cut"


from _utils import (
    apply_postprocessors,
    rejoin_formatted_num,
    strip_whitespace,
)
# from khmernlp.util.trie import Trie, dict_trie


def word_tokenize(
    text: str,
    custom_dict: Trie = Trie([]),
    engine: str = DEFAULT_WORD_TOKENIZE_ENGINE,
    keep_whitespace: bool = True,
    join_broken_num: bool = True,
) -> List[str]:

    if not text or not isinstance(text, str):
        return []

    segments = []


    if engine in ("mm", "multi_cut"):
        # from pythainlp.tokenize.multi_cut import segment
        # import segment
        segments = segment(text, custom_dict)


    else:
        raise ValueError(
            f"""Tokenizer \"{engine}\" not found.
            It might be a typo; if not, please consult our document."""
        )

    postprocessors = []
    if join_broken_num:
        postprocessors.append(rejoin_formatted_num)

    if not keep_whitespace:
        postprocessors.append(strip_whitespace)

    segments = apply_postprocessors(segments, postprocessors)

    return segments


class Tokenizer:


    def __init__(
        self,
        custom_dict: Union[Trie, Iterable[str], str] = [],
        engine: str = "multi_cut",
        keep_whitespace: bool = True,
        join_broken_num: bool = True,
    ):

        self.__trie_dict = Trie([])
        if custom_dict:
            self.__trie_dict = dict_trie(custom_dict)
        else:
            self.__trie_dict = DEFAULT_WORD_DICT_TRIE
        self.__engine = engine
        if self.__engine not in ["newmm", "mm", "longest", "deepcut", "multi_cut"]:
            raise NotImplementedError(
                """
                The Tokenizer class is not support %s for custom tokenizer
                """
                % self.__engine
            )
        self.__keep_whitespace = keep_whitespace
        self.__join_broken_num = join_broken_num

    def word_tokenize(self, text: str) -> List[str]:
        """
        Main tokenization function.

        :param str text: text to be tokenized
        :return: list of words, tokenized from the text
        :rtype: list[str]
        """
        return word_tokenize(
            text,
            custom_dict=self.__trie_dict,
            engine=self.__engine,
            keep_whitespace=self.__keep_whitespace,
            join_broken_num=self.__join_broken_num,
        )

    def set_tokenize_engine(self, engine: str) -> None:
        """
        Set the tokenizer's engine.

        :param str engine: choose between different options of tokenizer engines
                           (i.e. *newmm*, *mm*, *longest*, *deepcut*)
        """
        self.__engine = engine


### Compare Khmercut with multicut

In [26]:
text = """


បើតាមសេចក្តីប្រកាស​ព័ត៌មានមួយ​របស់​ក្រសួងការបរទេស​កម្ពុជា លោកវ៉ាង យី នឹងចូល​ក្រាប​បង្គំ​គាល់​ព្រះមហាក្សត្រ​ព្រះបាទ​សម្តេច​នរោត្តម សីហមុនី​ នៅ​ព្រះបរម​រាជវាំង។

ប្រមុខការទូត​ចិនរូបនេះ​ក៏គ្រោង​ចូល​ជួប​សម្តែង​ការ​គួរសម​ដាច់ដោយ​ឡែក​ជាមួយប្រធាន​ព្រឹទ្ធ​សភា លោក ហ៊ុន​ សែន និង​នាយករដ្ឋមន្ត្រី​លោក ហ៊ុន ម៉ាណែត។ ក្រៅ​ពីជំនួបនេះ​ លោក​វ៉ាង យី​ ក៏​នឹង​ជួបពិភាក្សា​ជាមួយសម​ភាគីរបស់​លោក​គឺ​រដ្ឋមន្ត្រី​ការបរទេស​និង​កិច្ចសហប្រតិបត្តិការ​អន្ត​រជាតិ​ លោក​សុខ ចិន្តា​សោភា​។

អំឡុងពេល​មាន​វត្តមាន​នៅប្រទេស​កម្ពុជា លោក វ៉ាង យី​ នឹង​ធ្វើជាសហប្រធាន​ជាមួយ​ឧប​នាយក​រដ្ឋមន្ត្រី​ និង​ជា​អនុប្រធានទី​១ ក្រុមប្រឹក្សា​អភិវឌ្ឍន៍កម្ពុជា ដឹកនាំ​កិច្ចប្រជុំ​គណៈកម្មាធិការ​សម្រប​សម្រួល​អន្តររដ្ឋាភិបាល​កម្ពុជា-ចិន​លើក​ទី​៧។ នេះបើយោងតាម​ក្រសួងការបរទេសកម្ពុជា។​

ដំណើរ​ទស្សនកិច្ច​របស់​លោក វ៉ាង យីមក​កាន់​ប្រទេស​កម្ពុជា​នៅពេល​នេះ ត្រូវអ្នកវិភាគ​មួយ​ចំនួន​មើល​ឃើញថា​ ជាការពង្រឹង​ទំនាក់​ទំនងរវាង​រដ្ឋាភិបាល​ថ្មីនៃប្រទេស​កម្ពុជា និង​រដ្ឋាភិបាល​ក្រុង​ប៉េកាំង ហើយ​ក៏ជា​សាទរ​ចំពោះអតីត​នាយករដ្ឋមន្ត្រីលោក​ហ៊ុន សែន​ដែល​ទើបតែ​បានកា្លយ​ជា​ប្រធាន​ព្រឹទ្ធសភា​នាពេល​ថ្មីៗនេះ​។





"""

In [29]:
# segment(text)

ignore_words = [
    '\n', 
    ' ', 
    '  ', 
    '\u200b', 
    '\t', 
    '\r'
]

correct_words = {
    "ពិេ សស": "ពិេសស",
    "ពីេសស": "ពិេសស",
    "ខ្បាល": "ក្បាល",
    "កម្ពុ": "កម្ពុជា",
}

custom_words = [
    "ខ្បាល",
    "គណអធីបតី",
    "១០",
    "ណែរនាំ",
    "P60",
    "កម្ពុជា", # still error on word "កម្ពុជា"
    "ថេប្លេត",
    "Galaxy",
    "S23",
    "Ultra"
    "S9+",
    "S9",
    "សំខាន់ៗ​",
    "​មជ្ឈិម​បក្ស​កុម្មុយ​និស្ត",
    "កុម្មុយ​និស្ត",
    "សេចក្តី",
    "អន្តរជាតិ",
    "បានក្លាយជា"
    
]

custome_words_list = set(khmer_words())
# add multiple words to the dictionary
custome_words_list.update(custom_words)

trie = dict_trie(dict_source=custome_words_list)

custom_tokenizer = Tokenizer(custom_dict=trie, engine="multi_cut")

list_results = custom_tokenizer.word_tokenize(text)


for word in list_results:
    if word not in ignore_words:
        if word in correct_words:
            print(correct_words[word])
        else:
            print(word)

បើតាម
សេចក្តី
ប្រកាស
ព័ត៌មាន
មួយ
របស់
ក្រសួង
ការបរទេស
កម្ពុជា
លោក
វ៉ាង
យី
នឹង
ចូល
ក្រាប
បង្គំ
គាល់
ព្រះមហាក្សត្រ
ព្រះបាទ
សម្តេច
នរោត្តម
សីហមុនី
នៅ
ព្រះ
បរម
រាជវាំង
។
ប្រមុខ
ការទូត
ចិន
រូបនេះ
ក៏
គ្រោង
ចូល
ជួប
សម
្
តែង
ការ
គួរសម
ដាច់
ដោយ
ឡែក
ជាមួយ
ប្រធាន
ព្រឹទ្ធ
សភា
លោក
ហ៊ុន
សែន
និង
នាយករដ្ឋមន្ត្រី
លោក
ហ៊ុន
ម៉ាណែត
។
ក្រៅ
ពី
ជំនួប
នេះ
លោក
វ៉ាង
យី
ក៏
នឹង
ជួប
ពិភាក្សា
ជាមួយសម
ភាគី
របស់
លោក
គឺ
រដ្ឋមន្ត្រី
ការបរទេស
និង
កិច្ច
សហប្រតិបត្តិការ
អន្ត
រ
ជាតិ
លោក
សុខ
ចិន្តា
សោភា
​។
អំឡុងពេល
មាន
វត្តមាន
នៅ
ប្រទេស
កម្ពុជា
លោក
វ៉ាង
យី
នឹង
ធ្វើជាសហប្រធាន
ជាមួយ
ឧ
ប
នាយក
រដ្ឋមន្ត្រី
និង
ជា
អនុប្រធាន
ទី
១
ក្រុមប្រឹក្សា
អភិវឌ្ឍន៍
កម្ពុជា
ដឹកនាំ
កិច្ចប្រជុំ
គណៈកម្មាធិការ
សម្រប
សម្រួល
អន្តររដ្ឋាភិបាល
កម្ពុជា
-
ចិន
លើក
ទី
៧
។
នេះ
បើយោងតាម
ក្រសួង
ការបរទេសកម្ពុជា
។​
ដំណើរ
ទស្សនកិច្ច
របស់
លោក
វ៉ាង
យីមក
កាន់
ប្រទេស
កម្ពុជា
នៅពេល
នេះ
ត្រូវ
អ្នក
វិភាគ
មួយ
ចំនួន
មើល
ឃើញថា
ជា
ការ
ពង្រឹង
ទំនាក់
ទំនងរវាង
រដ្ឋាភិបាល
ថ្មី
នៃ
ប្រទេស
កម្ពុជា
និង
រដ្ឋាភិបាល
ក្រុង
ប៉េកាំង
ហើយ
ក៏
ជា
សាទរ
ចំពោះ
អតីត
នាយករដ្ឋមន្ត្រី
លោក
ហ៊ុន
សែន

#### khmercut

In [17]:
from khmercut import tokenize

# tokenize(text)

ignore_words = [
    '\n',
    '\n\n',
    '\n\n\n',
    '',
    ' ', 
    '  ', 
    '\u200b', 
    '\t', 
    '\r'
]

for word in tokenize(text):
    if not word in ignore_words:
        print(word)


ព័ត៌មាន
នៅ
ថ្ងៃ
នេះ
មាន
ដូចជា
បទសម្ភាសន៍
VOA
អំពី
​ជន
​រងគ្រោះ
នៃ
​របប
ប្រល័យ
ពូជសាសន៍
ចែក
រំលែក
បទពិសោធន៍
​តស៊ូ
នៅ
អាមេរិក
និង
អតីតកាល
។
បទសម្ភាសន៍
វីអូអេ
ជាមួយ
នឹង
លោក
វេជ្ជបណ្ឌិត
សៀង
សេង
នៅ
រដ្ឋ
California
អំពី
​«
ជំងឺ
លើស
ឈាម
»
និង
ព័ត៌មាន
ផ្សេង
ទៀត
៕





